In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation
%matplotlib ipympl
import numpy as np
import pandas as pd

from scipy.spatial import KDTree, cKDTree
from scipy.ndimage import uniform_filter1d
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import time
import joblib
from tqdm import tqdm
import trackpy as tp
from numba import njit, prange

from yupi import Trajectory
import yupi.graphics as yg
import yupi.stats as ys

from utility import get_imsd, get_imsd_windowed, get_emsd, get_emsd_windowed, fit_hist, MB_2D,\
                    normal_distr, get_trajs, speed_windowed, theta_windowed, get_smooth_trajs, get_velocities
                   
show_verb = True
save_verb = True
anim_show_verb = False

In [2]:
rawTrajs = pd.read_parquet("../tracking/results/parquet/pre_merge_tracking.parquet")
red_particle_idx = 17
rawTrajs.loc[rawTrajs.particle != red_particle_idx, ["color"]] = "#00007F"
rawTrajs.loc[rawTrajs.particle == red_particle_idx, ["color"]] = "#FF0000"
colors = rawTrajs.loc[rawTrajs.frame == 0, 'color'].values
nDrops = len(rawTrajs.loc[rawTrajs.frame==0])
nFrames = max(rawTrajs.frame) + 1
print(f"nDrops:{nDrops}")
print(f"nFrames:{nFrames} --> {nFrames/10:.2f} s")


# WINDOWED ANALYSIS PARAMETERS
window = 3200 # 320 s
stride = 100 # 10 s
print(f"window of {window/10} s, stride of {stride/10} s")
startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window
nSteps = len(startFrames)
print(f"number of steps: {nSteps}")


# step 10 with a 10 fps video --> 1 s
units = "px/s"
default_kwargs_blue = {"color": "#00FFFF", "ec": (0, 0, 0, 0.6), "density": True}
default_kwargs_red = {"color": "#EE4B2B", "ec": (0, 0, 0, 0.6), "density": True}

nDrops:50
nFrames:32000 --> 3200.00 s
window of 320.0 s, stride of 10.0 s
number of steps: 288


In [3]:
# Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)

# MSD

In [4]:
show_verb = False
run_windowed_analysis = False
animated_plot_verb = False
print(f"MSD Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")

%run ./analysis_modules/msd.ipynb

MSD Analysis: show_verb = False, run_windowed_analysis = False, animated_plot_verb = False
Global IMSD
Global EMSD
Raw trajs - Blue Particles: 1.179 ± 0.002, Red Particle: 0.762 ± 0.002
Smooth trajs - Blue Particles: 1.183 ± 0.002, Red Particle: 0.765 ± 0.002
Windowed IMSD
Windowed EMSD


# SPEED & TURNING ANGLES DISTRIBUTIONS

In [ ]:
show_verb = False
animated_plot_verb = False
print(f"Speed and Turning Angles Analysis: show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/speed_and_turning_angles.ipynb

# VELOCITY AUTOCORRELATION

In [5]:
show_verb = False
run_windowed_analysis = False
overwrite = False
animated_plot_verb = False

print(f"Velocity Autocorrelation Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_autocorrelation.ipynb

Velocity Autocorrelation Analysis: show_verb = False, run_windowed_analysis = False, animated_plot_verb = False
Global Velocity Autocovariance Function
Windowed analysis


# RADIAL DISTRIBUTON FUNCTION

In [6]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

print("Radial Distribution Function Analysis: run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/radial_distribution_function.ipynb

Radial Distribution Function Analysis: run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}
Radial Distribution Function 
Radial Distribution Function from the center


# 3 BODY STRUCTURE METRIC

In [7]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

%run ./analysis_modules/3body_analysis.ipynb

Calculating 3-body angles for raw trajs...


100%|██████████| 32000/32000 [00:15<00:00, 2022.12it/s]


Calculating 3-body angles for smooth trajs...


100%|██████████| 32000/32000 [00:14<00:00, 2170.02it/s]


[[-4.91950863e-04 -2.10295746e-01  3.74798520e-02 ... -2.80754088e-02
   1.08862700e-01  5.74505769e-01]
 [-1.21492192e-01  9.32149025e-02  1.71536941e+00 ...  2.46996235e-02
   8.81826506e-02  1.26986255e-02]
 [ 1.28812997e+00  7.69975378e-02 -2.16473331e-03 ... -8.47483882e-01
  -8.25670375e-03 -5.19737686e-02]
 ...
 [ 1.06039114e-01  7.16752066e-03  3.22019303e-02 ... -7.32807325e-03
  -1.35886755e-01 -3.97214909e-02]
 [-1.07390152e-01 -2.01982371e-01 -7.34379684e-02 ...  1.01658457e-01
   9.24140708e-02  5.55032744e-02]
 [-1.48274284e-01 -6.71211350e-02  1.61650982e-01 ... -8.98000434e-02
   1.35891397e-02 -1.70256258e-01]]
